# Recursive Retriever


In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

file_path = "../data/paper/stanford-cynicism.pdf"

## Extracting PDFs and Tables


Here is a great [doc](https://unstract.com/blog/extract-tables-from-pdf-python/) for comparing different table extraction techniques.

For PDFPlumber, here is reference for [table extraction](https://github.com/jsvine/pdfplumber?tab=readme-ov-file#extracting-tables)


In [3]:
from pathlib import Path

print(Path().cwd())
Path("../data/paper/stanford-cynicism.pdf").exists()

/Users/pmui/SynologyDrive/research/2025/research2025/06-advanced-rag/notebooks


True

In [4]:
import pdfplumber

tables = []
with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        table = page.extract_table(
            table_settings={
                "horizontal_strategy": "lines_strict",
                "vertical_strategy": "lines_strict",
                "intersection_tolerance": 10,
            }
        )
        if table:
            tables.append(table)

len(tables)

1

In [5]:
from IPython.display import display, HTML
display(tables[0])

[['Year', '# Opinion Articles'],
 ['2010', '506'],
 ['2011', '482'],
 ['2012', '457'],
 ['2013', '340'],
 ['2014', '455'],
 ['2015', '447'],
 ['2016', '408'],
 ['2017', '427'],
 ['2018', '367'],
 ['2019', '328'],
 ['2020', '379'],
 ['2021', '187'],
 ['2022', '132'],
 ['2023', '106'],
 ['2024', '92']]

In [6]:
import pandas as pd

def process_table(table):
    if not table or len(table) == 0:
        return None
    
    # Convert to DataFrame first without headers
    df = pd.DataFrame(table)
    
    # Check if first row looks like headers
    first_row = df.iloc[0]
    
    # Heuristics to detect if first row is header:
    # 1. Check if first row is different data type than rest
    # 2. Check if first row has string-like values
    # 3. Check if first row doesn't contain numeric values
    is_header = all(isinstance(val, str) and not str(val).replace('.','').isdigit() 
                   for val in first_row if val is not None)
    
    if is_header:
        # Use first row as headers, drop the row, and reset index
        headers = [str(col).strip() for col in df.iloc[0]]
        df = df[1:]  # Remove the header row
        df.columns = headers
    else:
        # If no header, use default column names
        df.columns = [f'Column_{i+1}' for i in range(len(df.columns))]
    
    # Reset index after modifications
    df = df.reset_index(drop=True)
    
    # Clean up any None or empty string values
    df = df.replace(['', 'None', None], pd.NA)
    
    return df

In [35]:
import pandas as pd
# Process all tables
table_dfs = []
for i, table in enumerate(tables):
    df = process_table(table)
    if df is not None:
        table_dfs.append(df)
        
print(table_dfs)

[    Year # Opinion Articles
0   2010                506
1   2011                482
2   2012                457
3   2013                340
4   2014                455
5   2015                447
6   2016                408
7   2017                427
8   2018                367
9   2019                328
10  2020                379
11  2021                187
12  2022                132
13  2023                106
14  2024                 92]


## Pandas Query Engines


In [47]:
df.columns

Index(['Year', '# Opinion Articles'], dtype='object')

In [51]:
from llama_index.experimental.query_engine import PandasQueryEngine
llm = OpenAI(model="gpt-4o")

df_query_engines = [
    PandasQueryEngine(
        df, llm=llm, verbose=True,
    ) for df in table_dfs
]
len(df_query_engines)

1

In [54]:
response = df_query_engines[0].query("How many articles do you have in '2011' of type string?")
response.response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[df['Year'] == '2011']['# Opinion Articles'].sum()
```
> Pandas Output: 482


'482'

In [55]:
df['# Opinion Articles']

0     506
1     482
2     457
3     340
4     455
5     447
6     408
7     427
8     367
9     328
10    379
11    187
12    132
13    106
14     92
Name: # Opinion Articles, dtype: object